In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta
import pickle
import requests
import numpy as np
from glicko2 import Glicko2
from trueskill import TrueSkill
from scipy.stats import norm
from collections import defaultdict
%matplotlib inline

In [ ]:
df = pd.read_csv('data.csv')

## First look at the data

This is real Counter-Strike match data.

`perid` is the Bayes Esports identifier

- Take a look at the available data points
- Check data integrity
- How would you restructure it? (Hint: Think about the possible usecases)

## Elo

In [ ]:
# write two functions: one that calculates the expected win probability and one that updates the rating
# remember you also need an initial rating value

def expected() -> float:
    pass

def update() -> float:
    pass

In [ ]:
# now let's apply that function to the data!

In [ ]:
# plotting time: let's look at the team with the most ratings in our dataset

## Testing Elo quality

- We don't need a test set
- Careful about calibration time

In [ ]:
# write a function for the brier score

def brier() -> float:
    pass

In [ ]:
# you want to use the elo scores BEFORE the match to predict the outcome, then compare to the actual outcome

In [ ]:
# tuning time!

## Glicko-2

In [ ]:
# parameters to play around with
tau = .2
sigma = .06
mu = 1500
phi = 350
glicko = Glicko2(mu=mu, phi=phi, tau=tau, sigma=sigma)
# glicko.create_rating(mu=mu, phi=phi)
# glicko.rate(old_rating, series)
# series is a list of ([result, opponent_rating]) tuples

In [ ]:
# which period do you want to use as series? Day, Week, Month...
# remember to have provisional ratings within the rating period and only update the "real rating" after the series end!

#### pseudocode (don't read if you want a challenge)

    for period_name, series in df.groupby(period):
    
        provisional_ratings = {}
    
        for team in all_teams:
    
            - get all matches the team has played in this series
               
            - if so, rewrite team results in the format taken by algorithm
        
            - calculate new provisional rating of the team using this series and save it
    
        after doing this for all teams, the provisional ratings you calculated become the real ratings of the teams

In [ ]:
# win predictions
def predict_glicko_winner(r1: Glicko2.rating, r2: Glicko2.rating) -> float:
    r1_g2 = glicko.scale_down(r1)
    r2_g2 = glicko.scale_down(r2)
    # equivalent to calling Glicko2.reduce_impact
    g = 1 / np.sqrt(1+3*r1_g2.phi **2 / np.pi**2)
    # equivalent to calling Glicko2.expect_score
    E = 1 / (1 + np.exp(-g * (r1_g2.mu - r2_g2.mu)))
    return E
    

## Trueskill

In [ ]:
mu = 25
sigma = 8.33
p_draw = 0
beta = 8.333 / 2
ts = TrueSkill(backend='scipy')
# ts.Rating(mu=mu, sigma=sigma)
# ts.rate_1vs1(winner_rating, loser_rating)

In [ ]:
# win predictions
# TrueSkill provides a function (quality()) to calculate a draw probability between arbitrary ratings (because matchmaking is the goal)
# But there’s no function for a win probability.

def ts_win_probability(r1: TrueSkill.rating, r2: TrueSkill.rating):
    delta_mu = r1.mu - r2.mu
    sigmas = np.array([r1.sigma, r2.sigma, np.sqrt(2)*beta])
    p_1 = norm.cdf(delta_mu / np.linalg.norm(sigmas))
    p_2 = 1 - p_1
    ## correct for draws
    #p_1 -= 0.5 * p_draw
    #p_2 -= 0.5 * p_draw
    #if p_1 < 0:
    #    p_1 = 0
    #    p_2 += np.abs(p_1)
    #if p_2 < 0:
    #    p2 = 0
    #    p_1 += np.abs(p_2)
    return p_1, p_2, p_draw
